In [ ]:
%matplotlib notebook

In [ ]:
https://mybinder.org/v2/gh/samaloney/rhessi20.git/HEAD

# stixpy
* Aim to be interface for STIX data and analysis in python
* net - provides search, download, capabilities through `Fido` interface
* timeseries - load, plot and interact with quick look data
* science - load, plot and interact with science data

# stixpy
* Code - https://github.com/samaloney/stixpy
* Documentaion - https://stixpy.readthedocs.io/en/latest/index.html
* Very early in developement
  * subject to large changes
  * bugs 100% present
* Help wanted and appreciated - doesn't have to be code, issues, bug reports, questions are all welcome and useful but so is code!

## Imports
 * Import the functions or objects we need
 * Register or make available specific clients (e.g. for STIX)

In [ ]:
import numpy as np
from sunpy.net import Fido, attrs as a
from sunpy.timeseries import TimeSeries

from stixpy.net.client import STIXClient  # Registers the STIXClient with Fido
from stixpy import timeseries # Registers the STIX timeseries with TimeSeries
from stixpy.science import ScienceData

## Searching for data with Fido
* Unified search interface across a number of packages (sunpy, radiospecta, heliopy, stixpy) and data providers (VSO, JSOC, NOAA, ...) more info [here](https://docs.sunpy.org/en/stable/guide/acquiring_data/fido.html)
* Powerful query capabilities and result filtering
* Auto complete for discovery

## Searching for data with Fido

In [ ]:
a.Instrument

## Searching for data with Fido

In [ ]:
a.stix.DataType

## Searching for data with Fido

In [ ]:
query = Fido.search(a.Time('2021-05-06T00:00', '2021-05-08T23:59'),
                    a.Instrument.stix, a.stix.DataType.ql)

## Searching for data with Fido

In [ ]:
query

## Searching  for quick look data with Fido

In [ ]:
ql_lc_query =  Fido.search(a.Time('2021-05-06T00:00', '2021-05-08T23:59'),
                           a.Instrument.stix,
                           a.stix.DataProduct.ql_lightcurve)
ql_lc_query

## Downloading quick look data with Fido

In [ ]:
ql_lc_res = Fido.fetch(ql_lc_query)
ql_lc_res

## stixpy.timeseries
* Built on `sunpy.timeseries`
* Focus on quick look data and currently supports
  * Quick look light curves
  * Quick look background
  * Quick look variance
* For more information check the docs [here](https://stixpy.readthedocs.io/en/latest/code_ref/timeseries.html)

## Loading and plotting quick look data

In [ ]:
ql_lc = TimeSeries(ql_lc_res)
ql_lc

In [ ]:
ql_lc[0].plot();

## Loading and plotting quick look data +
* Can do a better job by concatenating all the individual timeseries together `concatenate=True`

In [ ]:
ql_lc_combined = TimeSeries(ql_lc_res, concatenate=True)
ql_lc_combined

## Load and plot

In [ ]:
ql_lc_combined.plot();

## An aside - SolarOrbiter constraints and STIX operations
 * SOLO is a deep space mission - telemetry limited (or at leas the budget for TM is limited)
 * All instruments optimised to meet science requirements while meeting TM constraints
 * Spectrograms relatively cheap on TM (sum 32 x 12 detector and pixels on-board)
 * Most if not all flares will have at least Spectrograms

## stixpy.science
* Science data products
  * RawPixelData - essentially a dump of on the on-board archive (times x detectors x 12 x energies)
  * CompressedPixelData - compress on board-archive (times x detectors x 12 x energies)
  * SummedCompressedPixelData - nominally sum from 12 to 4 pixels and compress (times x detectors x 4 x energies)

## stixpy.science
* Science data products
  * Visibilities - sum pixel and subtract (A-C, B-D) and compress (times x 2 x energies)
  * Spectrogram - sum over detectors and pixels and compress (times x energies)
  

## stixpy.science
 * Share common interface to data and plotting
 * `plot_pixels` - pixel data only, plot of individual pixels
 * `plot_timeseries` - sum over detector and pixel and energies
 * `plot_spectrogram` - sum over detectors and pixels
 * `get_data` - all method to extract the underling data (counts, time, triggers, ...)
 * For more information check the documentation [here](https://stixpy.readthedocs.io/en/latest/code_ref/science.html)

## Searching for Spectrograms with Fido

In [ ]:
sci_query =  Fido.search(a.Time('2021-05-07T18:30', '2021-05-07T20:00'),
                         a.Instrument.stix, a.stix.DataProduct.sci_xray_spec)
sci_query

## Downloading Spectrograms with Fido

In [ ]:
spec_dl = Fido.fetch(sci_query[0][1])
spec_dl

## Load Spectrogram

In [ ]:
spec = ScienceData.from_fits(spec_dl[0])
type(spec)

In [ ]:
spec

###  Plot Spectrogram

In [ ]:
spec.plot_spectrogram();

In [ ]:
tids = (np.arange(0, 601)*5)
ti = np.vstack([tids[:-1], tids[1:]]).T
spec.plot_spectrogram(energy_indices=range(1,28), time_indices=ti)
                     #e.g. time_indices = [[0, 5][5, 10], [10, 15]]

## Searching for PixelData with Fido

In [ ]:
pixel_query =  Fido.search(a.Time('2021-05-07T18:30', '2021-05-07T20:00'),
                           a.Instrument.stix, a.stix.DataProduct.sci_xray_cpd)
pixel_query

## Downloading Spectrograms with Fido

In [ ]:
pixel_dl = Fido.fetch(pixel_query[0][-2])
pixel_dl

## Load PixelData

In [ ]:
cpd = ScienceData.from_fits(pixel_dl[0])
type(cpd)

In [ ]:
cpd

In [ ]:
cpd.plot_spectrogram();

In [ ]:
cpd.plot_pixels()